
原本的捲積模型


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# 定義一個簡單的卷積神經網路
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 設置資料轉換
transform = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

# 加載FashionMNIST資料集
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# 初始化模型、損失函數和優化器
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練模型
model.train()
for epoch in range(10):  # 假設訓練10個epoch
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print("訓練完成！")

# 評估模型
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'測試集上的準確率: {100 * correct / total}%')

100%|██████████| 26421880/26421880 [00:01<00:00, 16755229.02it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 274914.63it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5067320.64it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 19057614.29it/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

訓練完成！
測試集上的準確率: 91.62%


主要變更

數據集：使用datasets.MNIST來替換datasets.FashionMNIST。

輸出層：將全連接層的輸出數改為10，對應MNIST數據集中0到9的數字分類。
主要調整

**增加卷積層數量和通道數**

更深的特徵提取：增加卷積層的數量可以讓模型學習到更高層次、更抽象的特徵。這對於識別複雜模式和細節非常有幫助。

更多的通道：增加每個卷積層的通道數可以讓模型在每一層中學習到更多的特徵，從而提高模型的表達能力。

**增加全連接層的神經元數量**

更強的分類能力：增加全連接層的神經元數量可以提高模型的分類能力，因為更多的神經元可以捕捉到更多的特徵組合，從而更好地進行分類。

**調整批次大小**

穩定的梯度更新：增加批次大小可以使梯度更新更加穩定，從而有助於模型更好地收斂。同時，較大的批次大小可以更充分地利用GPU的計算能力，提升訓練效率。

減少噪聲：較大的批次大小可以減少梯度更新中的隨機噪聲，使模型的訓練過程更加平滑。

**調整學習率**

更穩定的訓練過程：降低學習率可以使模型訓練過程更加穩定，減少震盪和過度擬合的風險。特別是在模型變得更深、更複雜的情況下，適當降低學習率可以幫助模型更好地收斂。

更精細的權重調整：較低的學習率允許模型在每一步中進行更精細的權重調整，有助於找到更優的解。

**增加訓練epoch數**

更充分的訓練：增加訓練epoch數可以讓模型有更多的機會學習數據中的模式和特徵，從而提高模型的性能。

更好的泛化能力：更多的訓練epoch可以幫助模型更好地擬合訓練數據，從而提高在測試數據上的表現。

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# 定義一個更深的卷積神經網路
class DeeperCNN(nn.Module):
    def __init__(self):
        super(DeeperCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(256 * 3 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)  # MNIST有10個類別（數字0-9）

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 256 * 3 * 3)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 設置資料轉換
transform = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

# 加載MNIST資料集
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)  # 調整批次大小
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)  # 調整批次大小

# 初始化模型、損失函數和優化器
model = DeeperCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # 調整學習率

# 訓練模型
model.train()
for epoch in range(20):  # 增加訓練epoch數
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print("訓練完成！")

# 評估模型
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'測試集上的準確率: {100 * correct / total}%')

訓練完成！
測試集上的準確率: 99.18%
